In [11]:
import jax.numpy as jnp
import jax
import numpy as np
import equinox
import torch
import locor.test as affine_transformation_torch
import locor
from locor import affine_transformation

In [20]:
source_data_test = torch.randn(4, 12)
source_data_test_numpy = source_data_test.numpy()

In [27]:
affine_torch = affine_transformation_torch.generate_affine_transformation_matrix(
    parameters=source_data_test,
    transformation_type=affine_transformation_torch.AffineTransformationTypeDefinition.full(),
    inverse=True,
)
affine = affine_transformation.generate_affine_transformation_matrix(
    parameters=source_data_test_numpy,
    transformation_type=affine_transformation.AffineTransformationTypeDefinition.full(),
    inverse=True,
)
print(np.amax(np.abs(affine_torch.numpy() - affine)))
print(affine_torch[0])
print(affine[0])

3.8146973e-06
tensor([[ 2.4881, -1.7256, -2.7512,  1.0949],
        [-3.3787,  4.5073,  0.2748,  1.6547],
        [ 1.7935, -0.2718, -1.4170,  1.6879],
        [ 0.0000,  0.0000,  0.0000,  1.0000]])
[[ 2.4881127  -1.725615   -2.751151    1.0949339 ]
 [-3.3787146   4.5072684   0.27476966  1.6546583 ]
 [ 1.7935138  -0.27182332 -1.4169611   1.6878794 ]
 [ 0.          0.          0.          1.        ]]


In [8]:
sampler = cm.CubicSplineSampler()
coordinate_system = cm.CoordinateSystem.voxel(
    spatial_shape=source_data_test.shape[2:],
)
source_test = cm.samplable_volume(source_data_test, coordinate_system=coordinate_system)
target_test = cm.samplable_volume(target_data_test, coordinate_system=coordinate_system)

loss = local_least_squares_error(source_test, target_test, sampler, coordinate_system, regularization=1)
print(loss.shape)
print(float(loss.mean()))

torch.Size([1, 1, 16, 16, 16])
-0.4502831697463989


In [9]:
import jaxmorph as jm
import locor
import locor.local_least_squares_error
with jax.default_device(jax.devices("cpu")[0]):
    sampler = jm.CubicSplineSampler()
    coordinate_system = jm.CoordinateSystem.voxel(
        spatial_shape=source_data_test.shape[2:],
    )
    source_test = jm.samplable_volume(source_data_test_numpy, coordinate_system=coordinate_system)
    target_test = jm.samplable_volume(target_data_test_numpy, coordinate_system=coordinate_system)

    loss = locor.local_least_squares_error.local_least_squares_error(source_test, target_test, sampler, coordinate_system, regularization=1)
    print(loss.shape)
    print(float(loss.mean()))

(1, 1, 16, 16, 16)
-0.45028313994407654
